In [13]:
from langchain_community.llms import Ollama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain

from utils import print_info

llm = Ollama(model="llama2")

In [14]:
# # combine prompt and llm into simple LLM chain
# prompt = ChatPromptTemplate.from_messages([
#     ("system", "You are world class technical documentation writer."),
#     ("user", "{input}")
# ])

# # add simple output parser to convert the chat message to a string.
# output_parser = StrOutputParser()
# chain = prompt | llm | output_parser
# chain.invoke({"input": "how can langsmith help with testing?"})


In [15]:
# Retrieval Chain
# Retrieval is useful when you have too much data to pass to the LLM directly. 
# You can then use a retriever to fetch only the most relevant pieces and pass those in.
#(1) populate a vector store and use that as a retriever

loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()

#(2) index it into a vectorstore. This requires a few components, namely an embedding model and a vectorstore.
embeddings = OllamaEmbeddings()

#(3) use this embedding model to ingest documents into a vectorstore. use a simple local vectorstore, FAISS to build our index
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

#This chain will take an incoming question, look up relevant documents, then pass those documents 
# along with the original question into an LLM and ask it to answer the original question.

#(4)set up the chain that takes a question and the retrieved documents and generates an answer.
prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [16]:
from langchain_core.documents import Document

document_chain.invoke({
    "input": "how can langsmith help with testing?",
    "context": [Document(page_content="langsmith can let you visualize test results")]
})

'\nBased on the provided context, Langsmith can help with testing by allowing users to visualize their test results. This means that Langsmith provides a way to display and analyze the results of tests in a visual format, which can be useful for understanding the outcome of tests and identifying any issues or patterns.'

In [20]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

LangSmith can help with testing by providing a variety of features that make it easier to evaluate and test changes to prompts or chains. Some of the ways LangSmith can help with testing include:

1. Automatic evaluation metrics: LangSmith provides automatic evaluation metrics for getting consistent and scalable information about model behavior. However, human review is still necessary to get the utmost quality and reliability from your application.
2. Annotation queues: LangSmith allows you to manually review and annotate runs through annotation queues. You can select any runs based on criteria like model type or automatic evaluation scores and queue them up for human review.
3. Evaluators: LangSmith provides evaluators that can be specified when initiating a test run, and will evaluate the results once the test run completes. While these evaluators are not perfect, they can guide your eye to examples you should look at and help validate the results of automatic evaluation metrics.
4.